# 🧠 Day 11 – Image Classification with Transfer Learning

 📌 In this project, we aim to:
 - Use a pretrained CNN (like VGG16 or ResNet50) for image classification.
 - Learn how to freeze layers and fine-tune top layers.
 - Load image data using ImageDataGenerator.
 - Evaluate performance using accuracy, loss, and confusion matrix.
 ✅ We’ll use a subset of the Dogs vs Cats dataset.


In [15]:
# ✅ Step 1: Import Required Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
print("Current working directory:", os.getcwd())
print("Folders in current directory:")
print(os.listdir())

Current working directory: d:\Programming\Machine-Learning\Day 11
Folders in current directory:
['cats_and_dogs_filtered', 'Transfer Learning.ipynb']


In [14]:
# Correct base directory (based on confirmed structure)
base_dir = os.path.join(os.getcwd(), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Confirm folder contents
print("✅ Train Cats:", os.listdir(os.path.join(train_dir, 'cats'))[:5])
print("✅ Train Dogs:", os.listdir(os.path.join(train_dir, 'dogs'))[:5])


✅ Train Cats: ['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg']
✅ Train Dogs: ['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg']


In [ ]:
# Image size and batch size
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

# Training data generator with rescaling (can augmentation here later)
train_datagen = ImageDataGenerator(rescale=1./255)

# Validation data generator just rescales
val_datagen = ImageDataGenerator(rescale=1./255)

# Create train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'   # because we have two classes (cats and dogs)
)

# Create validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load the VGG16 model, excluding the top classification layer
base_model = VGG16(
    weights='imagenet',         # Use pretrained weights
    include_top=False,          # Do NOT include the dense layers at the top
    input_shape=(160, 160, 3)   # Input shape must match our resized images
)

# Freeze the entire base model so its weights don't change during training
base_model.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [17]:
# Get the output of the base model
x = base_model.output

# Flatten the 3D feature maps into 1D
x = Flatten()(x)

# Add a fully connected (dense) layer
x = Dense(256, activation='relu')(x)

# Add dropout for regularization (prevents overfitting)
x = Dropout(0.5)(x)

# Add the final output layer (binary classification: cat or dog)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)
